# [Module 2.2] 로컬 모드 및 스크립트 모드로 훈련 (SageMaker Hugging Face 사용)

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 환경 셋업
- 2. 세이지 메이크 로컬 모드 훈련
- 3. SageMaker Host Mode 로 훈련
- 4. 실험 결과 보기
- 5. 모델 아티펙트 저장

---

참고:

- Use Hugging Face with Amazon SageMaker
    - 세이지 메이커 개발자 문서
    - https://docs.aws.amazon.com/sagemaker/latest/dg/hugging-face.html


- 세이지 메이커로 파이토치 사용 
    - [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)


- Use PyTorch with the SageMaker Python SDK
    - https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html


- Amazon SageMaker Local Mode Examples
    - TF, Pytorch, SKLean, SKLearn Processing JOb에 대한 로컬 모드 샘플
        - https://github.com/aws-samples/amazon-sagemaker-local-mode
    - Pytorch 로컬 모드
        - https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/pytorch_script_mode_local_training_and_serving/pytorch_script_mode_local_training_and_serving.py    



# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
### 커스텀 라이브러리
import config 

버킷 및 폴더(prefix) 설정

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "KoElectra-HF"



# 2. 세이지 메이크 로컬 모드 훈련
#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [4]:
import os
import subprocess

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Mon Jul 11 13:32:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2.1. 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 2.2.훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [5]:
train_code = 'src/train_Trainer.py'
!pygmentize {train_code}

import argparse
import os
import json
import sys

from train_lib import train_Trainer

def parser_args():
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=128)
    parser.add_argument("--test_batch_size", type=int, default=256)    
    parser.add_argument("--learning_rate", type=float, default=5e-5)
    parser.add_argument("--warmup_steps", type=int, default=0)    
    parser.add_argument("--weight_decay", type=float, default=0.01)           
    parser.add_argument("--fp16", type=lambda x: (str(x).lower() == 'true'), default=True)    
    parser.add_argument("--disable_tqdm", type=lambda x: (str(x).lower() == 'true'), default=True)        
    parser.add_argument("--tokenizer_id", type=str, default='monologg/koelectra-small-v3-discriminator')
    parser.add_argument("--model_id", type=st

## 2.3. 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [6]:
train_local_inputs = config.train_data_dir
test_local_inputs = config.test_data_dir
print("train_local_inputs: ", train_local_inputs)
print("test_local_inputs: ", test_local_inputs)


train_local_inputs:  data/nsmc/train
test_local_inputs:  data/nsmc/test


In [7]:
local_inputs = {
                'train': f'file://{train_local_inputs}',
                'test': f'file://{test_local_inputs}',    
               }

## 2.4. 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [8]:
hyperparameters = {'epochs': 1, 
                   'train_batch_size' : 256,
                   'eval_batch_size' : 256,  
                   'test_batch_size' : 256,                                                              
                   'learning_rate': 5e-5,
                   'warmup_steps' : 0,
                   'weight_decay' : 0.01,
                   'fp16' : "True",
                   'tokenizer_id' : 'monologg/koelectra-small-v3-discriminator',
                   'model_id' : 'monologg/koelectra-small-v3-discriminator',     
                   'is_evaluation' : "True",
                   'is_test' : "True",
                   'eval_ratio' : 0.5,
                   'use_subset_train_sampler' : "False",
                   'disable_tqdm' : 'True',
                   'logging_steps' : 100,
                   'seed' : 100
                    }  

In [9]:
from sagemaker.huggingface import HuggingFace
from sagemaker.pytorch import PyTorch
import os
import subprocess
import time

job_name = f'huggingface-native-pytorch-190-hol-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# local_estimator = PyTorch(
#     entry_point="train_Trainer.py",    
#     source_dir='src',    
#     role=role,
#     framework_version='1.9.1',
#     py_version='py38',
#     instance_count=1,
#     instance_type=instance_type, # local_gpu or local 지정
#     session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
#     hyperparameters= hyperparameters               
    
# )

local_estimator = HuggingFace(
    entry_point="train_Trainer.py",    
    source_dir='src',    
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = 1,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version  = '4.11.0',           # the transformers version used in the training job
    pytorch_version       = '1.9.0',            # the pytorch_version version used in the training job
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.    
    disable_profiler      = True,
    debugger_hook_config  = False, 
#     checkpoint_s3_uri     = chkpt_s3_path,
#     checkpoint_local_path ='/opt/ml/checkpoints',  
)



local_estimator.fit(local_inputs)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Creating nd7wbe9gvb-algo-1-2ewh2 ... 
Creating nd7wbe9gvb-algo-1-2ewh2 ... done
Attaching to nd7wbe9gvb-algo-1-2ewh2
nd7wbe9gvb-algo-1-2ewh2 | 2022-07-11 13:37:07,297 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
nd7wbe9gvb-algo-1-2ewh2 | 2022-07-11 13:37:07,323 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
nd7wbe9gvb-algo-1-2ewh2 | 2022-07-11 13:37:07,327 sagemaker_pytorch_container.training INFO     Invoking user training script.
nd7wbe9gvb-algo-1-2ewh2 | 2022-07-11 13:37:07,751 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
nd7wbe9gvb-algo-1-2ewh2 | /opt/conda/bin/python3.8 -m pip install -r requirements.txt
nd7wbe9gvb-algo-1-2ewh2 | Collecting torch==1.9.1
nd7wbe9gvb-algo-1-2ewh2 |   Downloading torch-1.9.1-cp38-cp38-manylinux1_x86_64.whl (831.4 MB)
     |████████████████████████████████| 831.4 MB 1.8 kB/s  eta 0:00:01
nd7wbe9gvb-algo-1-2ewh2 |

# 3. SageMaker Host Mode 로 훈련
- instance_type, session 을 수정 합니다.
- 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
- wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
- 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

## 3.1. 데이터 세트를 S3에 업로드


In [10]:
s3_train_data_loc = sagemaker_session.upload_data(path=config.train_data_dir, bucket=bucket, 
                                       key_prefix=f"{prefix}/{config.train_data_dir}")

s3_test_data_loc = sagemaker_session.upload_data(path=config.test_data_dir, bucket=bucket, 
                                       key_prefix=f"{prefix}/{config.test_data_dir}")

print("s3_train_data_loc: ", s3_train_data_loc)
print("s3_test_data_loc: ", s3_test_data_loc)

s3_train_data_loc:  s3://sagemaker-us-east-1-280862185298/KoElectra-HF/data/nsmc/train
s3_test_data_loc:  s3://sagemaker-us-east-1-280862185298/KoElectra-HF/data/nsmc/test


In [11]:
! aws s3 ls {s3_train_data_loc} --recursive
! aws s3 ls {s3_test_data_loc} --recursive

2022-07-11 13:41:01   13296169 KoElectra-HF/data/nsmc/train/ratings_train.txt
2022-07-11 13:41:01    4449062 KoElectra-HF/data/nsmc/test/ratings_test.txt


## 3.2. 훈련 및 테스트 데이터를 S3 로 지정

In [12]:
s3_inputs = {
            'train': f'{s3_train_data_loc}',
            'test': f'{s3_test_data_loc}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-280862185298/KoElectra-HF/data/nsmc/train', 'test': 's3://sagemaker-us-east-1-280862185298/KoElectra-HF/data/nsmc/test'}


## 3.3. 실험 세팅

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 1_Setup_Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


### 실험 생성

In [13]:
# !pip install --upgrade sagemaker-experiments
import boto3
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm_client = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = prefix 

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for NCF',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:KoElectra-HF is created


### [중요] 하이퍼 파라미터 
- epochs 
    - 값을 조절해서 실행 시간을 조정 하세요.
- `'use_subset_train_sampler' : False,`
    - False 이면 전체 훈련 데이터를 사용
    - True 이면 전체 훈련 데이터의 0.01 만큼만 사용 (빠른 코드 디버깅시 사용)
- `'is_evaluation' : True,`
    - True 이면 전체 훈련 데이터에서 `eval_ratio` (에: 0.2) 데이타를 쪼개어서 사용 
        - `eval_ratio` 가 0.2 이면 train --> train 80% : val 20 % 로 쪼개짐.
    - False 이면 전체 훈련 데이터 사용
- `is_test` 가 True 이면 테스트 데이터로 모델 평가 함.

In [14]:
hyperparameters = {'epochs': 1, 
                   'train_batch_size' : 256,
                   'eval_batch_size' : 256,  
                   'test_batch_size' : 256,                                                              
                   'learning_rate': 5e-5,
                   'warmup_steps' : 0,
                   'weight_decay' : 0.01,
                   'fp16' : "True",
                   'tokenizer_id' : 'monologg/koelectra-small-v3-discriminator',
                   'model_id' : 'monologg/koelectra-small-v3-discriminator',     
                   'is_evaluation' : "True",
                   'is_test' : "True",
                   'eval_ratio' : 0.1,
                   'use_subset_train_sampler' : "False",
                   'disable_tqdm' : 'True',
                   'logging_steps' : 100,
                   'seed' : 100
                    }  

### 시도(Trial) 생성

In [15]:
from datetime import datetime
# 시도 이름 생성
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
trial_name = experiment_name + f"-{ts}"

# 1개의 실험 안에 시도를 생성함.
response = sm_client.create_trial(
    ExperimentName = experiment_name,
    TrialName = trial_name,
)    

# 실험 설정: 실험 이름, 시도 이름으로 구성
experiment_config = {
    'ExperimentName' : experiment_name,
    'TrialName' : trial_name,
    "TrialComponentDisplayName" : 'Training',
}    



## 3.4 훈련 실행


### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [16]:
metric_definitions=[
       {'Name': 'Accuracy', 'Regex': 'Acc=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}        
    ]


In [18]:
# chkpt_s3_path = f's3://{sess.default_bucket()}/{s3_prefix}/native/checkpoints'
import time

from sagemaker.huggingface import HuggingFace
instance_type = 'ml.p3.2xlarge'
instance_count = 1
job_name = f'huggingface-native-pytorch-190-hol-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'


# num_gpus = 4

# batch_size_native = 32
# learning_rate_native = float('5e-5')
# batch_size = batch_size_native
# learning_rate = learning_rate_native / batch_size_native * batch_size * num_gpus * instance_count
# logger.info(learning_rate)

# create the Estimator
host_estimator = HuggingFace(
    entry_point="train_Trainer.py",    
    source_dir='src',    
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = instance_count,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version  = '4.11.0',           # the transformers version used in the training job
    pytorch_version       = '1.9.0',            # the pytorch_version version used in the training job
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler      = True,
    debugger_hook_config  = False, 
#     checkpoint_s3_uri     = chkpt_s3_path,
#     checkpoint_local_path ='/opt/ml/checkpoints',  
)

host_estimator.fit(s3_inputs, 
                   experiment_config = experiment_config, # 실험 설정 제공                   
                   wait=False)

INFO:sagemaker:Creating training-job with name: huggingface-native-pytorch-190-hol-2022-2022-07-11-13-43-28-403


In [19]:
%%time

host_estimator.logs()

2022-07-11 13:43:29 Starting - Starting the training job...
2022-07-11 13:43:54 Starting - Preparing the instances for training.........
2022-07-11 13:45:32 Downloading - Downloading input data
2022-07-11 13:45:32 Training - Downloading the training image..............................
2022-07-11 13:50:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-11 13:50:27,203 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-11 13:50:27,223 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-11 13:50:27,230 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-11 13:50:27,758 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
  Attempting uninst

# 4. 실험 결과 보기


위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [20]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,disable_tqdm,epochs,eval_batch_size,eval_ratio,fp16,is_evaluation,is_test,learning_rate,logging_steps,model_id,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,seed,test_batch_size,tokenizer_id,train_batch_size,use_subset_train_sampler,warmup_steps,weight_decay,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,huggingface-native-pytorch-190-hol-2022-2022-0...,Training,arn:aws:sagemaker:us-east-1:280862185298:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/h...,1.0,ml.p3.2xlarge,30.0,"""True""",5.0,256.0,0.1,"""True""","""True""","""True""",0.00005,100.0,"""monologg/koelectra-small-v3-discriminator""",20.0,"""huggingface-native-pytorch-190-hol-2022-2022-...","""train_Trainer.py""","""us-east-1""","""s3://sagemaker-us-east-1-280862185298/hugging...",100.0,256.0,"""monologg/koelectra-small-v3-discriminator""",256.0,"""False""",0.0,0.01,None,s3://sagemaker-us-east-1-280862185298/KoElectr...,None,s3://sagemaker-us-east-1-280862185298/KoElectr...,None,s3://sagemaker-us-east-1-280862185298/huggingf...,[KoElectra-HF-2022-07-11-13-41-02-246364],[KoElectra-HF]


### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [21]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.Accuracy.max",        
    sort_order="Descending",
    metric_names=["Accuracy"],    
    parameter_names=["epochs", "train_batch_size",
                    ],
)

trial_component_training_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,epochs,train_batch_size,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,huggingface-native-pytorch-190-hol-2022-2022-0...,Training,arn:aws:sagemaker:us-east-1:280862185298:train...,5.0,256.0,None,s3://sagemaker-us-east-1-280862185298/KoElectr...,None,s3://sagemaker-us-east-1-280862185298/KoElectr...,None,s3://sagemaker-us-east-1-280862185298/huggingf...,[KoElectra-HF-2022-07-11-13-41-02-246364],[KoElectra-HF]


# 5. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [22]:
trainer_artifact_path = host_estimator.model_data
print("trainer_artifact_path: ", trainer_artifact_path)

%store trainer_artifact_path

trainer_artifact_path:  s3://sagemaker-us-east-1-280862185298/huggingface-native-pytorch-190-hol-2022-2022-07-11-13-43-28-403/output/model.tar.gz
Stored 'trainer_artifact_path' (str)


기타 변수 저장

In [23]:
%store bucket 
%store prefix

Stored 'bucket' (str)
Stored 'prefix' (str)
